# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [2]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [4]:
class TextToBinary:
    def __init__(self, source):
        self.source = source

    def read(self, amount):
        return self.source.read(amount).encode()

    def readline(self, amount=-1):
        return self.source.readline(amount).encode()


def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    with open(filename, 'r') as f:
        data = pickle.load(TextToBinary(f))
    
    target = [i] * len(data)
    return data, target

In [5]:
# load all the data
all_data = []
all_target = []
# MY CODE
# Loop through each class index to aggregate data and targets
for index in range(0, len(classes)):
    # Load the pose data for the current class index
    (data_chunk, target_chunk) = load_pose_data(index)

    # Aggregate the data chunks to the main data collection
    all_data += data_chunk
    
    # Aggregate the target chunks to the main target collection
    all_target += target_chunk

print ('total number of data', len(all_data))

total number of data 222


In [6]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [7]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [8]:
# YOUR CODE HERE
# Convert lists to NumPy arrays if they aren't already
feature_set = np.asarray(all_data)
target_set = np.asarray(all_target)

clf.fit(feature_set[training_data], target_set[training_data])

SVC(C=100.0, gamma=0.001)

### predicting

In [9]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])

In [10]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [11]:
expected_data = []
predicted_data = []
# YOUR CODE HERE
# Retrieve the true labels of the training data subset for evaluation
expected_data = target_set[training_data]

# # Use the trained classifier to predict labels for the same training data subset
predicted_data = clf.predict(feature_set[training_data])

# # Evaluate the classifier's performance by comparing expected and predicted labels
evaluate(expected_data, predicted_data)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00         6
           4       1.00      1.00      1.00         7
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00        12
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00        19
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        36

    accuracy                           1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [12]:
expected = []
predicted = []

expected = target_set[training_data]
predicted = clf.predict(feature_set[training_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00         6
           4       1.00      1.00      1.00         7
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00        12
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00        19
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        36

    accuracy                           1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0 16  0  0  0  0  0  0  0  0  0]
 [ 0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [13]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [14]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict([feature_set[-1]]), target_set[-1]

(array([10]), 10)